# 의사결정나무 모델
## 분류 나무(Classification Tree)
- 종속변수가 __명목형__ 인 경우 사용하는 의사결정나무 모델
- 각 노드 분류 알고리즘은 이진 분류 시 지니지수(Gini Index) 기반의 CART 사용
- 과적합 방지 및 모델 단순화를 위해 Depth, Impurity등 관련 설정 활용

## 회귀 나무(Regression Tree)
- 종속변수가 __연속형__ 인 경우 사용하는 의사결정나무 모델
- 각 노드 분류에는 평균, 표준편차 활용
- 과적합 방지 및 모델 단순화를 위해 Depth, Impurity등 관련 설정 활용

## 주요 함수 및 메서드
__sklearn-DecisionTreeClassifier()__

__sklearn-DecisionTreeRegressor()__

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

In [2]:
df = pd.read_csv("Data/iris.csv")

In [4]:
df["is_setosa"] = (df["Species"] == "setosa") + 0
df.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,is_setosa
0,5.1,3.5,1.4,0.2,setosa,1
1,4.9,3.0,1.4,0.2,setosa,1


In [5]:
model_c = DecisionTreeClassifier(random_state=123)
model_c.fit(X = df.iloc[:, :4],
            y = df["is_setosa"])
model_c

DecisionTreeClassifier(random_state=123)

In [6]:
pred_c = model_c.predict(df.iloc[:, :4])
pred_c[:4]

array([1, 1, 1, 1])

In [8]:
# 3개 이상일 때는 predict_proba
# pred_c = model_c.predict_proba(df.iloc[:, :4])
# pred_c

In [9]:
model_r = DecisionTreeRegressor(random_state=123)
model_r.fit(X = df.iloc[:, :3],
            y = df["Petal.Width"])
pred_r = model_r.predict(df.iloc[:, :3])
pred_r[:4]

array([0.25, 0.2 , 0.2 , 0.2 ])

# 문제

In [10]:
# ====================================================
# 1. 당뇨병 발병 여부를 예측하기 위하여 의사결정나무를 사용하고자 한다.
# 이 때 혈당, 혈압, 임신 횟수를 기반으로 예측을 했을 때 예측 정확도는?

# 정답: 63%
# ====================================================
df = pd.read_csv("Data/diabetes.csv")

In [11]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, train_size=0.8, random_state = 123)

In [12]:
model = DecisionTreeClassifier(random_state = 123).fit(X = df_train.loc[:, ["Glucose", "BloodPressure", "Pregnancies"]],
                                                       y = df_train["Outcome"])
pred = model.predict(df_test.loc[:, ["Glucose", "BloodPressure", "Pregnancies"]])
pred[:4]

array([1, 0, 1, 0], dtype=int64)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_score(y_pred = pred, y_true = df_test["Outcome"])

0.6298701298701299

In [16]:
# ====================================================
# 2. 환자의 BMI를 예측하기 위하여 회귀나무 사용.
# 이때 혈당, 혈압, 피부 두께를 독립변수로 했을 경우 RMSE?
# train:test = 8:2
# seed = 123

# 정답: 9.9
# ====================================================
df = pd.read_csv("Data/diabetes.csv")

In [17]:
df_train, df_test = train_test_split(df, train_size=0.8, random_state = 123)

In [19]:
model = DecisionTreeRegressor(random_state = 123).fit(X = df_train.loc[:, ["Glucose", "BloodPressure", "SkinThickness"]],
                                                      y = df_train["BMI"])
pred = model.predict(df_test.loc[:, ["Glucose", "BloodPressure", "SkinThickness"]])
pred[:4]

array([30.8, 28.6, 33.8, 32.2])

In [20]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_pred = pred, y_true = df_test["BMI"]) ** 0.5

9.924605211306321

In [21]:
# ====================================================
# 3. 분류나무의 파라미터를 바꿔가면서 성능 평가
# 당뇨 발병 여부를 종속변수로 하고 
# 혈당, 혈압, 임신 횟수, BMI, 나이를 독립변수로 하고
# Depth를 3에서 6까지 변화시킬 때 그 결과로 틀린 것은?
# train:test = 7:3
# seed = 345

# 1) Depth 3, 정확도: 0.77
# 2) Depth 4, 정확도: 0.76
# 3) Depth 5, 정확도: 0.76
# 4) Depth 6, 정확도: 0.78
# ====================================================
df = pd.read_csv("Data/diabetes.csv")

In [22]:
df_train, df_test = train_test_split(df, train_size=0.7, random_state = 345)

In [24]:
cols = ["Glucose", "BloodPressure", "Pregnancies", "BMI", "Age"]

depth_list = [3, 4, 5, 6]
accs = []
for depth in depth_list:
    model = DecisionTreeClassifier(max_depth = depth,
                                  random_state = 345)
    model.fit(X = df_train.loc[:, cols],
              y = df_train["Outcome"])
    pred = model.predict(df_test.loc[:, cols])
    accs = accs + [accuracy_score(y_pred = pred, y_true = df_test["Outcome"])]
    
df_score = pd.DataFrame({"depth": depth_list,
                         "accs": accs})
df_score["accs"] = df_score["accs"].round(2)
df_score

,depth,accs
0,3,0.77
1,4,0.76
2,5,0.76
3,6,0.77
